In [ ]:
import os
import zipfile
import shutil
import re

# Путь к папке с архивами
cubemaps_dir = 'cubemaps'
# Путь к папке, куда будут распакованы файлы
output_dir = 'data'

# --- Подготовительные действия ---
# Создаем папку для распакованных файлов, если она еще не существует
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# --- Основной цикл ---
# Проходим по всем файлам в папке с архивами
for filename in os.listdir(cubemaps_dir):
    if not filename.endswith('.zip'):
        continue

    # --- Извлечение номера 'n' из имени архива ---
    # Случай для "Standard-Cube-Map.zip"
    if filename == 'Standard-Cube-Map.zip':
        n = 0
    else:
        # Используем регулярное выражение для извлечения числа из скобок, например, (1), (2), ... (12)
        match = re.search(r'\((\d+)\)', filename)
        if match:
            n = int(match.group(1))
        else:
            # Пропускаем файлы .zip, которые не соответствуют шаблону
            continue

    # --- Создание папки для распаковки ---
    new_dir_path = os.path.join(output_dir, str(n))
    if not os.path.exists(new_dir_path):
        os.makedirs(new_dir_path)

    # --- Распаковка и переименование ---
    zip_path = os.path.join(cubemaps_dir, filename)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for member in zip_ref.namelist():
            # Получаем чистое имя файла, например "nx.png"
            original_name = os.path.basename(member)

            # Пропускаем системные файлы или папки внутри архива
            if not original_name or not original_name.endswith('.png'):
                continue
            
            # **ИСПРАВЛЕННАЯ ЛОГИКА**
            prefix_char = original_name[0]  # Первый символ: 'n' или 'p'
            axis_char = original_name[1]    # Второй символ: 'x', 'y' или 'z'

            # Определяем, 'neg' или 'pos'
            side = 'neg' if prefix_char == 'n' else 'pos'
            
            # Формируем новое имя файла
            new_name = f"{n}_{side}{axis_char}.png"
            target_path = os.path.join(new_dir_path, new_name)

            # Извлекаем файл и сохраняем его с новым именем
            with zip_ref.open(member) as source, open(target_path, "wb") as target:
                shutil.copyfileobj(source, target)

print("Распаковка и переименование завершены.")

In [7]:
import pyvista as pv
import numpy as np
import os
import random
import math

class STLRenderer:
    """
    A class to render STL files with randomized backgrounds, object colors, textures,
    and adaptive camera positioning.
    """
    def __init__(self):
        # --- Object appearance options ---
        self.object_colors = [
            '#C0C0C0', '#A9A9A9', '#808080', # Silvers & Greys
            '#F5F5DC', '#FFE4B5', '#DEB887', # Beiges & Woods
            '#4682B4', '#5F9EA0', '#87CEEB', # Blues & Teals
            '#90EE90', '#3CB371', '#2E8B57', # Greens
            '#E0FFFF', '#ADD8E6'             # Light Blues
        ]
        self.background_options = ["black", "grey", "#303030", None]

        # --- Cubemap options ---
        self.cubemap_loaders = [
            pv.examples.download_cubemap_park,
            pv.examples.download_cubemap_space_4k,
            pv.examples.download_sky_box_cube_map,
        ]
        if self.cubemap_loaders:
            print(f"Found {len(self.cubemap_loaders)} available cubemaps.")

    def generate_render(self, stl_path: str, output_path: str, resolution: tuple[int, int],
                        camera_position: list, mesh: pv.PolyData,
                        background_color: str | list | None = None,
                        cubemap_texture: pv.Texture | None = None,
                        object_color: str = 'white',
                        object_texture: pv.Texture | None = None):
        """
        Generates a single render with specified appearance for object and background.
        """
        plotter = pv.Plotter(off_screen=True, window_size=resolution)
        # plotter.enable_pbr() # Must be enabled for realistic materials and textures

        mesh.texture_map_to_plane(inplace=True)
        plotter.add_mesh(
            mesh,
            color=object_color,
            texture=object_texture,
            smooth_shading=True,
            metallic=0.3,
            roughness=0.5
        )

        if cubemap_texture:
            plotter.add_actor(cubemap_texture.to_skybox())
            plotter.set_environment_texture(cubemap_texture)
        else:
            plotter.set_background(background_color)

        plotter.camera_position = camera_position
        plotter.screenshot(output_path, transparent_background=(background_color is None and not cubemap_texture))
        plotter.close()

    def generate_random_renders(self, stl_path: str, output_dir: str, num_renders: int,
                                resolution: tuple[int, int], textures_dir: str | None = None):
        """
        Orchestrates generating multiple random renders with adaptive camera and appearance.
        """
        if not os.path.exists(stl_path):
            print(f"Error: STL file not found at {stl_path}")
            return

        # Load mesh once and center it. All renders will use this centered mesh.
        mesh = pv.read(stl_path)
        mesh.translate(-np.array(mesh.center), inplace=True)

        # Find available textures
        texture_files = []
        if textures_dir and os.path.exists(textures_dir):
            texture_files = [os.path.join(textures_dir, f) for f in os.listdir(textures_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        os.makedirs(output_dir, exist_ok=True)
        base_name = os.path.splitext(os.path.basename(stl_path))[0]
        
        print(f"\nGenerating {num_renders} random renders for {base_name}.stl...")
        print(f"Model diagonal length: {mesh.length:.2f}. Camera distance will be adapted.")
        if texture_files: print(f"Found {len(texture_files)} textures.")

        for i in range(num_renders):
            # --- 1. Randomize Object Appearance (Color vs Texture) ---
            use_texture = random.choice([True, False]) and texture_files
            obj_color, obj_texture = 'white', None
            
            if use_texture:
                texture_path = random.choice(texture_files)
                obj_texture = pv.read_texture(texture_path)
                appearance_log = f"Texture: {os.path.basename(texture_path)}"
            else:
                obj_color = random.choice(self.object_colors)
                appearance_log = f"Color: {obj_color}"

            # --- 2. Randomize Background (Cubemap vs Monochrome) ---
            use_cubemap = random.choice([True, False]) and self.cubemap_loaders
            bg_color, cubemap, bg_log = None, None, ""

            if use_cubemap:
                loader_func = random.choice(self.cubemap_loaders)
                cubemap = loader_func()
                bg_log = f"Cubemap: {loader_func.__name__}"
            else:
                bg_color = random.choice(self.background_options)
                bg_log = "transparent" if bg_color is None else str(bg_color)
            
            # --- 3. Set Camera Position based on object size ---
            camera_pos = get_random_camera_position(mesh, relative_distance_range=(2.5, 4.5))
            
            # --- 4. Generate and Save ---
            output_filename = os.path.join(output_dir, f"{base_name}_{i + 1}.png")
            self.generate_render(
                stl_path=stl_path,
                output_path=output_filename,
                resolution=resolution,
                camera_position=camera_pos,
                mesh=mesh.copy(), # Pass a copy to avoid any potential interference
                background_color=bg_color,
                cubemap_texture=cubemap,
                object_color=obj_color,
                object_texture=obj_texture
            )
            print(f"  -> Saved {output_filename} (Object: {appearance_log} | BG: {bg_log})")

def get_random_camera_position(mesh: pv.PolyData, relative_distance_range: tuple[float, float]) -> list:
    """Generates a random camera position adapted to the mesh's size."""
    # Calculate absolute distance based on the mesh's diagonal length
    model_size = mesh.length
    distance = random.uniform(*relative_distance_range) * model_size
    
    # If model is very small, prevent camera from being too close
    if distance < 1.0: distance = 1.0

    azimuth = random.uniform(0, 360)
    elevation = random.uniform(-90, 90)

    elev_rad, azim_rad = math.radians(90 - elevation), math.radians(azimuth)
    x = distance * math.sin(elev_rad) * math.cos(azim_rad)
    y = distance * math.sin(elev_rad) * math.sin(azim_rad)
    z = distance * math.cos(elev_rad)

    return [(x, y, z), [0, 0, 0], [0, 0, 1]]

if __name__ == "__main__":
    # --- Configuration ---
    STL_FILE = "sample_cube.stl"
    OUTPUT_DIR = "renders"
    TEXTURES_DIR = "textures" # <--- Put your texture images here
    NUM_RENDERS = 10
    RESOLUTION = (512, 512)

    # --- Setup for demonstration ---
    if not os.path.exists(STL_FILE):
        pv.Cube().save(STL_FILE)
        print(f"Created a sample file: {STL_FILE}")
    
    if not os.path.exists(TEXTURES_DIR):
        os.makedirs(TEXTURES_DIR)
        print(f"Created directory: {TEXTURES_DIR}")
        print(f"!!! PLEASE ADD YOUR .JPG/.PNG TEXTURE FILES TO THIS DIRECTORY !!!")

    # --- Main execution ---
    renderer = STLRenderer()
    renderer.generate_random_renders(
        stl_path=STL_FILE,
        output_dir=OUTPUT_DIR,
        num_renders=NUM_RENDERS,
        resolution=RESOLUTION,
        textures_dir=TEXTURES_DIR
    )
    print(f"\nSuccess! All renders saved in the '{OUTPUT_DIR}' directory.")

Found 3 available cubemaps.

Generating 10 random renders for sample_cube.stl...
Model diagonal length: 1.73. Camera distance will be adapted.
Found 10 textures.
  -> Saved renders/sample_cube_1.png (Object: Texture: Ground055S_2K-JPG_Color.jpg | BG: Cubemap: download_cubemap_space_4k)
  -> Saved renders/sample_cube_2.png (Object: Texture: Metal053B_2K-JPG_Color.jpg | BG: Cubemap: download_sky_box_cube_map)
  -> Saved renders/sample_cube_3.png (Object: Color: #87CEEB | BG: transparent)
  -> Saved renders/sample_cube_4.png (Object: Color: #C0C0C0 | BG: black)
  -> Saved renders/sample_cube_5.png (Object: Color: #2E8B57 | BG: transparent)
  -> Saved renders/sample_cube_6.png (Object: Texture: Metal058A_2K-JPG_Color.jpg | BG: Cubemap: download_sky_box_cube_map)
  -> Saved renders/sample_cube_7.png (Object: Color: #5F9EA0 | BG: #303030)
  -> Saved renders/sample_cube_8.png (Object: Color: #87CEEB | BG: Cubemap: download_cubemap_space_4k)
  -> Saved renders/sample_cube_9.png (Object: Color: